In [1]:
from scipy.special import wofz
import matplotlib.image as mpimg
from scipy.signal import savgol_filter
import fisspy
import fisspy.image.base as fissimage 
from fisspy.analysis.doppler import lambdameter
import numpy as np

In [ ]:
class fissDB:
    def __init__(self, fissHa, fissCa, fHapars=None, fCapars=None):
        try:
            plt.rcParams['keymap.back'].remove('left')
            plt.rcParams['keymap.forward'].remove('right')
        except:
            pass
        
        self.fissHa = fissHa
        self.fissCa = fissCa
        self.nx = self.fissHa.nx
        self.xDelt = self.fissHa.xDelt
        self.yDelt = self.fissHa.yDelt
        
        if self.fissHa.ny >= self.fissCa.ny:
            self.fissHa.data = self.fissHa.data[:self.fissCa.ny]
            self.ny = self.fissCa.ny
            self.extentRaster = self.fissCa.extentRaster
        elif fissHa.ny < fissCa.ny:
            self.fissCa.data = self.fissCa.data[:self.fissHa.ny]
            self.ny = self.fissHa.ny
            self.extentRaster = self.fissHa.extentRaster
        self._xMin = self.extentRaster[0]
        self._xMax = self.extentRaster[1]
        self._yMin = self.extentRaster[2]
        self._yMax = self.extentRaster[3]
        
        sh = alignoffset(self.fissCa.data[:,2:-2,50], self.fissHa.data[:,2:-2,-50])
        self.sh = sh
        print('sh=', sh)
        tmp = shift3d(fissCa.data.transpose(2, 0, 1), -sh).transpose(1,2,0)
        self.fissCa.data = tmp
        self.fissCa.data[tmp<10] = 1
        del tmp
        self.parHa = np.empty((16,self.ny,self.nx), float)
        self.Hapar = False
        if not (fHapars is None):
            self.Hapar = True
            Ha =fits.getheader(fHapars) 
            parA=fits.getdata(fHapars)
#            parA = np.flip(parA, 2)
            x1, x2, y1, y2 = Ha['XSTART'], Ha['NAXIS1']+Ha['XSTART'],  Ha['YSTART'], Ha['NAXIS2']+Ha['YSTART']       
            y2 = Leq(y2, self.ny)
            for p in range(16):
 #               print(self.parHa[p,y1:y2,x1:x2].sHape, (parA[p,:, :]*Ha[rf'SCALE{p:02d}']).sHape)
                self.parHa[p,y1:y2,x1:x2] = parA[p,0:y2-y1, 0:x2-x1]*Ha[rf'SCALE{p:02d}']

        self.parCa = np.empty((16,self.ny,self.nx), float)

        self.Capar = False
        if  not ( fCapars is None):
            self.Capar= True
            Ca =fits.getheader(fCapars) 
            par_Ca = fits.getdata(fCapars)
#            par_Ca = np.flip(par_Ca, 2)
            x1, x2, y1, y2 = Ca['XSTART'], Ca['NAXIS1']+Ca['XSTART'], Ca['YSTART'], Ca['NAXIS2']+Ca['YSTART']
            y2 = Leq(y2, self.ny)
            for p in range(16):
                tmp = np.zeros((self.ny, self.nx))
                tmp[y1:y2, x1:x2]=par_Ca[p,:, :]*Ca[rf'SCALE{p:02d}']
                self.parCa[p,:,:] = shift(tmp, -sh) 

        self.wvHaline = 6562.817
        self.wvCaline = 8542.091
        
    def Xpix2Mm(self, x, invert=False):
        if not invert:
            xnew = (x+0.5-self.nx/2.)*self.xDelt*0.725
        else:
            xnew = round(x/(self.xDelt*0.725)+self.nx/2-0.5)
        return xnew
    
    def Ypix2Mm(self, y, invert=False):
        if not invert:
            ynew = (y+0.5-self.ny/2.)*self.yDelt*0.725
        else:
            ynew = round(y/(self.yDelt*0.725)+self.ny/2-0.5)
        return ynew

    def imshow(self, x=None, y=None, wvHa=None, wvCa=None, hwHa=0.3,
               hwCa=0.2, scale='log', helpBox=True, imInterp='nearest'):
        """
        
        
        """
        self.imInterp = imInterp

        if not wvHa:
            wvHa = self.fissHa.centralWavelength
        if not wvCa:
            wvCa = self.fissCa.centralWavelength
        if not x:
            x = 0.  #self.nx//2
        if not y:
            y = 0. #self.ny//2
        xpix = self.Xpix2Mm(x, invert=True) #round(x/(self.xDelt*0.725)+self.nx/2-0.5)
        ypix = self.Ypix2Mm(y,invert=True) #round( y/(self.yDelt*0.725)+self.ny/2-0.5)
        self.xpix = xpix
        self.ypix = ypix
        self.x = self.Xpix2Mm(self.xpix) # (self.xpix+0.5-self.nx/2.)*self.xDelt*0.725 #(self.xpix+0.5)*self.xDelt*0.725
        self.y = self.Ypix2Mm(self.ypix)  #+0.5-self.ny/2.)*self.yDelt*0.725 #(self.ypix+0.5)*self.yDelt*0.725
        self.scale = scale
        self.hwHa = hwHa
        self.hwCa = hwCa
        self.wvHa = wvHa
        self.wvCa = wvCa
        self.xpixb = self.xpix0 = self.xpix
        self.ypixb = self.ypix0 = self.ypix
        self.xb = self.x0 = self.x
        self.yb = self.y0 = self.y
        self.wvHab = self.wvHa0 = self.wvHa
        self.wvCab = self.wvCa0 = self.wvCa
        self.xH = self.x
        self.yH = self.y
        self.wvHaH = self.wvHa
        self.wvCaH = self.wvCa
        helpBox = False            
        if helpBox:
            self._helpShow()
            
        self._iniFigure()
        
        if self.Hapar: self.figIRaster.canvas.mpl_connect('key_press_event', self._onKey)
        self.figProf.canvas.mpl_connect('key_press_event', self._onKey)
        self.figRaster.canvas.mpl_connect('key_press_event', self._onKey)
        self.figSpec.canvas.mpl_connect('key_press_event', self._onKey)
        
    def _iniFigure(self):
        HaWv = np.array([-0.5, 0, 0.5, -0.9, 4])
        CaWv = np.array([-0.25,0, 0.25, 0.7, 4])
        nWv = HaWv.size
        wf = 0.1   # angstrom to nm
        wfp = 100.  # asngstrom to pm
        lf =0.725 # arc second to Mm
 
        extent =np.append(self.Xpix2Mm(np.array([-0.5, self.nx-0.5]) ), 
                          self.Ypix2Mm(np.array([-0.5, self.ny-0.5 ])) )
 #       extent = np.array([(0-self.nx/2)*self.xDelt, (self.nx-self.nx/2)*self.xDelt,
 #                          (0-self.ny/2)*self.yDelt, (self.ny-self.ny/2)*self.yDelt])*lf
        self.fissHa.extentSpectro[2:4]= self.Ypix2Mm(np.array([-0.5, self.ny-0.5])) #np.array((0-self.ny/2)*self.yDelt, (self.ny-self.ny/2)*self.yDelt])*lf
        self.fissCa.extentSpectro[2:4]=  self.fissHa.extentSpectro[2:4]  
       # Raster
        self.figRaster, self.axRaster = plt.subplots(2, nWv,
                                                     num='Raster',
                                                     figsize=[self.nx/100.*nWv*1.2,5.*1.1],
                                                     sharex=True,
                                                     sharey=True,
                                                     clear=True)
        self.axRaster = self.axRaster.flatten()
        for n, hwv in enumerate(HaWv):
            raster = self.fissHa.getRaster(hwv, hw=0.05)
            wh = raster > 0.5*np.median(raster)
            if self.scale == 'log':
                raster = np.log10(raster)
            im = self.axRaster[n].imshow(raster,
                                         self.fissHa.cmap,
                                         origin='lower',
                                         extent=extent,
                                         interpolation=self.imInterp)
            m=raster[wh].mean()
            st =raster[wh].std()
            im.set_clim(m-3*st, m+2.*st)
            sg=''
            if hwv < 0.:  sg=rf'$-$'
            self.axRaster[n].set_title(rf'H$\alpha$: '+sg+rf'{wfp*abs(hwv):.0f} pm',fontsize=12)
#            self.axRaster[n].set_axis_off()            
        for n, cwv in enumerate(CaWv):
            raster = self.fissCa.getRaster(cwv, hw=0.03)
            wh = raster >  0.5*np.median(raster)
            if self.scale == 'log':
                raster = np.log10(raster)
            
            im = self.axRaster[n+nWv].imshow(raster,
                                           self.fissCa.cmap,
                                           origin='lower',
                                           extent=extent,
                                           interpolation=self.imInterp)
            m=raster[wh].mean()
            st =raster[wh].std()
            im.set_clim(m-3*st, m+2*st)
            sg=''
            if cwv < 0.:  sg=rf'$-$'
            self.axRaster[n+nWv].set_title(rf'Ca II: '+sg+rf'{wfp*abs(cwv):.0f} pm',fontsize=12)
            if n == 0:
                self.axRaster[n+nWv].set_xlabel('x (Mm)')
                self.axRaster[n+nWv].set_ylabel('y (Mm)')
                     
#  
#        self.vlineRaster = []
        self.position = []
        for ax in self.axRaster:
            self.position += [ax.scatter(self.x, self.y, 50, marker='+',
                                         color='c', linewidth=0.8)]
        self.figRaster.tight_layout(pad=1)
        
        
        #spectrogram
        self.figSpec = plt.figure('Spectrograph', figsize=[5, 6], clear=True)
        self.axSpecHa = self.figSpec.add_subplot(2, 1, 1)
        self.axSpecCa = self.figSpec.add_subplot(2, 1, 2)
        self.axSpecHa.set_xlabel(r'Wavelength ($\AA$)')
        self.axSpecHa.set_ylabel('Y (arcsec)')
        self.axSpecCa.set_xlabel(r'Wavelength ($\AA$)')
        self.axSpecCa.set_ylabel('Y (arcsec)')
        self.axSpecHa.set_title(rf"H$\alpha$ X={self.x:.2f}'', Y={self.y:.2f}''"
                                r" (X$_{pix}$="f"{self.xpix}, "
                                r"Y$_{pix}$="f"{self.ypix})")
        self.axSpecCa.set_title(rf"Ca II")
        specHa = self.fissHa.data[:, self.xpix]
        specCa = self.fissCa.data[:, self.xpix, ::-1]
        whHa = specHa > 5
        whCa = specCa > 5
        if self.scale == 'log':
            specHa = np.log10(specHa)
            specCa = np.log10(specCa)
        self.imSpecHa = self.axSpecHa.imshow(specHa, self.fissHa.cmap,
                                             origin='lower',
                                             extent=self.fissHa.extentSpectro,
                                             interpolation=self.imInterp)
        self.imSpecCa = self.axSpecCa.imshow(specCa, self.fissCa.cmap,
                                             origin='lower',
                                             extent=self.fissCa.extentSpectro,
                                             interpolation=self.imInterp)
        self.imSpecHa.set_clim(specHa[whHa].min(), specHa.max())
        self.imSpecCa.set_clim(specCa[whCa].min(), specCa.max())
        
        self.vlineSpecHa = self.axSpecHa.axvline(self.wvHa,
                                                 linestyle='dashed',
                                                 color='r', linewidth=0.5)
        self.vlineSpecCa = self.axSpecCa.axvline(self.wvCa,
                                                 linestyle='dashed',
                                                 color='r', linewidth=0.5)
        self.hlineSpecHa = self.axSpecHa.axhline(self.y,
                                                 linestyle='dashed',
                                                 color='r', linewidth=0.5)
        self.hlineSpecCa = self.axSpecCa.axhline(self.y,
                                                 linestyle='dashed',
                                                 color='r', linewidth=0.5)
        self.axSpecHa.set_aspect(adjustable='box', aspect='auto')
        self.axSpecCa.set_aspect(adjustable='box', aspect='auto')
        self.figSpec.tight_layout()
        

        wvA=self.fissHa.wave+self.wvHaline
        wvB=self.fissCa.wave+self.wvCaline
        
        profA0=self.fissHa.refProfile #/self.fissHa.refProfile.max()
        profB0=self.fissCa.refProfile  #/self.fissCa.refProfile.max()
        
        wvA = Recalibrate(wvA, profA0/profA0.max())
        self.fissHa.wave = wvA - self.wvHaline
        wvB = Recalibrate(wvB, profB0/profB0.max(), line='Ca')
        self.fissCa.wave = wvB - self.wvCaline
        wvA1=self.fissHa.wave
        wvA=wvA1+self.wvHaline
        wvB1=self.fissCa.wave
        wvB=wvB1+self.wvCaline

        pureA=fpure(wvA, line='Ha')
        profA = FissStray(pureA, wvA1, profA0, self.fissHa.refProfile)  
        parA,  modelA, modelpHa, fiterrorA, modelcHa \
            =  ThreeLayerModel(wvA, profA,  self.wvHaline, pureA, line='Ha')
        pureB=fpure(wvB, line='Ca')
        profB = FissStray(pureB, wvB1, profB0, self.fissCa.refProfile)  
        par_Ca,  modelB, modelpCa, fiterrorB, modelcCa \
            =  ThreeLayerModel(wvB, profB,  self.wvCaline, pureB, line='Ca')

                
        self.figProf =  plt.figure('Line Profile', figsize=[15*0.7,10*0.8],
                                   clear=True)
        axes = self.figProf.subplots(2,2, sharex='col', sharey='row')
        self.axProfHa = axes[0,0] #self.figProf.add_subplot(2, 2, 1)
        self.axProfCa = axes[0,1] #self.figProf.add_subplot(2, 2, 2)
        self.axProfHa1 = axes[1,0] # self.figProf.add_subplot(2, 2, 3)
        self.axProfCa1 = axes[1,1] #self.figProf.add_subplot(2, 2, 4)
       
      
        self.pProfHa = self.axProfHa.plot(wvA1*wfp,  profA,\
                    color='k', label="data")[0]
        self.pProfHaa = self.axProfHa.plot(wvA1*wfp,   modelpHa,\
                    color='r', linewidth=1., label="model ph")[0]
        self.pProfHab = self.axProfHa.plot(wvA1*wfp,   modelcHa,\
                    color='c', linewidth=1., label="model ch")[0]
        self.axProfHa.legend(loc='upper right')
        self.pProfHac=self.axProfHa.plot(wvA1*wfp, modelA,  color='g', linewidth=0.5)[0]

        self.axProfHa.set_ylabel('Intensity')
        self.axProfHa.set_yscale('log')
        self.axProfHa.plot(wvA1*wfp, profA, 
                           'b--', linewidth=1.5,
                           label='reference')[0]
 
        self.pProfCa = self.axProfCa.plot(wvB1*wfp,   profB, \
                      color='k', label="data")[0]
        self.pProfCaa = self.axProfCa.plot(wvB1*wfp,   modelpCa, \
                     color='r', linewidth=1., label="model ph")[0]
        self.pProfCab = self.axProfCa.plot(wvB1*wfp,   modelcCa, \
                     color='c', linewidth=1., label="model ch")[0]
        self.axProfCa.legend(loc='upper right') 
        self.pProfCac=self.axProfCa.plot(wvB1*wfp, modelB,  color='g', linewidth=0.5)[0]
        self.axProfCa.set_yscale('log')
        self.axProfCa.plot(wvB1*wfp,
                           profB,
                           'b--', linewidth=1.5,
                           label='reference')[0]
        self.pProfHa1a = self.axProfHa1.plot(wvA1*wfp,    profA/modelcHa-1,\
                    color='k', label="data")[0]
#        r1, r2 = fabsorp(wvA1, parA, self.wvHaline)
#        self.pProfHa1c = self.axProfHa1.plot(wvA1*wfp,  0.1*r1, \
#                      color='b', linewidth=1., label="r1")[0]
#        self.pProfHa1d = self.axProfHa1.plot(wvA1*wfp,  0.1*r2, \
#                      color='g', linewidth=1., label="r2")[0]        
        self.pProfHa1b = self.axProfHa1.plot(wvA1*wfp,  modelA/modelcHa-1.,\
                    color='r', linewidth=1., label="model")[0]

        self.pProfCa1a = self.axProfCa1.plot(wvB1*wfp,   profB/modelcCa-1, \
                      color='k', label="data")[0]
#        r1, r2 = fabsorp(wvB1, par_Ca, self.wvCaline)
#        self.pProfCa1c = self.axProfCa1.plot(wvB1*wfp,  0.1*r1, \
#                      color='b', linewidth=1., label="r1")[0]
#        self.pProfCa1d = self.axProfCa1.plot(wvB1*wfp,  0.1*r2, \
#                      color='g', linewidth=1., label="r2")[0]
        self.pProfCa1b = self.axProfCa1.plot(wvB1*wfp,  modelB/modelcCa-1., \
                      color='r', linewidth=1., label="model")[0]

#
        self.axProfHa1.set_xlabel(r'$\lambda-\lambda_0$ [pm]')
        self.axProfHa1.set_ylabel('Contrast')
        self.axProfCa1.set_xlabel(r'$\lambda-\lambda_0$ [pm]')
#        self.axProfCa1.set_ylabel('Contrast')
        self.axProfHa.set_ylim(0.05, 6.)
        self.axProfCa.set_ylim(0.05, 6.)
        ff=3.0
        wmin=-220*ff
        wmax=220*ff
        self.axProfHa.set_xlim(wmin, wmax)
        self.axProfCa.set_xlim(wmin, wmax)
        self.axProfHa1.set_xlim(wmin, wmax)
        self.axProfCa1.set_xlim(wmin, wmax)
        self.axProfHa1.set_ylim(-1.5,0.5)
        self.axProfCa1.set_ylim(-1.5,0.5)
        

        self.axProfHa.set_title(r'H$\alpha$')
        self.axProfCa.set_title(r'Ca II 854.2 nm')
        self.txtpHa = self.axProfHa.text(-200*ff, 0.1,    
                 rf'$x$={self.x:.1f}, $y$={self.y:.1f} Mm;  $v_p$={parA[0]:.2f} km/s;'+
                 rf'$S_p$={10**parA[4]:.2f}, $S_2$={10**(parA[5]):.2f}' ,fontsize=11  )
#           rf'$x$={self.x:.2f},$y$={self.y:.2f}; $v_p$={parA[0]:.2f} km/s; '+
#           rf'$S_p$={10**parA[4]:.2f}; $\delta_p$={parA[5]:.2f}'
        self.txtpHaa = self.axProfHa.text(-200*ff, 0.07,  
                rf'log $\eta$={parA[1]:.2f}, log $w_p$={parA[2]:.2f}; log $a$={parA[3]:.2f}'
                                                                         ,fontsize=11  ) 
#           rf'log $\eta$={parA[1]:.2f}, log $w_p$={parA[2]:.2f}; log $a$={parA[3]:.2f}',fontsize=11  )
        self.txtpHa1 = self.axProfHa1.text(-200*ff,-1.0,  
           rf'$v_2$={parA[8]:.2f}, $v_1$={parA[9]:.2f} km/s; $w_2$={100*10**(parA[10]):.0f}, '+
           rf'$w_1$={100*10.**parA[11]:.0f} pm',fontsize=11  ) 
#           rf'$v_2$={parA[7]:.2f}, $v_1$={parA[8]:.2f} km/s; $\delta_w$={parA[9]:.3f}, '+
#           rf'$w_1$={100*10.**parA[10]:.0f} pm',                    
#                               fontsize=11  )
        self.txtpHa1a = self.axProfHa1.text(-200*ff,-1.2,
           rf'$\tau_2$={10**parA[6]:.1f}, $\tau_1$={10**parA[7]:.1f}, $S_1$={10**parA[12]:.2f}, $S_0$={10**parA[13]:.2f}; $\epsilon$={fiterrorA:.3f}',fontsize=11  ) 
 #          rf'$S_1$={10.**parA[11]:.2f}, $\delta_1$={parA[12]:.2f}; $\epsilon$={fiterrorA:.3f}',                    
 #                              fontsize=11  )
        
        self.txtpCa = self.axProfCa.text(-200.*ff, 0.1, 
           rf'$x$={self.x:.1f}, $y$={self.y:.1f} Mm;  $v_p$={par_Ca[0]:.2f} km/s'+
           rf'; $S_p$={10**par_Ca[4]:.2f}, $S_2$={10**(par_Ca[5]):.2f} ',fontsize=11  ) 
#           rf'$x$={self.x:.2f},$y$={self.y:.2f}; $v_p$={par_Ca[0]:.2f} km/s; '+
#            rf'$S_p$={10**par_Ca[4]:.2f}, $\delta_p$={par_Ca[5]:.2f} '  ,fontsize=11  )
        self.txtpCaa = self.axProfCa.text(-200*ff, 0.07,  
           rf'log $\eta$={par_Ca[1]:.2f}, log $w_p$={par_Ca[2]:.2f}; log $a$={par_Ca[3]:.2f}',fontsize=11  ) 
#           rf'log $\eta$={par_Ca[1]:.2f}, log $w_p$={par_Ca[2]:.2f}; log $a$={par_Ca[3]:.2f}',fontsize=11  )
        self.txtpCa1 = self.axProfCa1.text(-200*ff, -1.0, 
            rf'$v_2$={par_Ca[8]:.2f}, $v_1$={par_Ca[9]:.2f} km/s; $w_2$={100*10.**(par_Ca[10]):.0f}, '+
            rf'$w_1$={1000*0.1*10.**par_Ca[11]:.0f} pm',fontsize=11  ) 
#           rf'$v_2$={par_Ca[7]:.2f}, $v_1$={par_Ca[8]:.2f} km/s; $\delta_w$={par_Ca[9]:.3f}, '+
#           rf'$w_1$={100*10.**par_Ca[10]:.0f} pm',  fontsize=11  )
        self.txtpCa1a = self.axProfCa1.text(-200*ff, -1.2, 
            rf'$\tau_2$={10**par_Ca[6]:.1f}, $\tau_1$={10**par_Ca[7]:.1f}, $S_1$={10**par_Ca[12]:.2f}, $S_0$={10**par_Ca[13]:.2f};  $\epsilon$={fiterrorB:.3f}',fontsize=11  ) 
 #          rf'$S_1$={10.**par_Ca[11]:.2f}, $\delta_1$={par_Ca[12]:.2f};  $\epsilon$={fiterrorB:.3f}',                    
 #                                 fontsize=11  )

        temp, xi=ftandxi(10.**parA[10], 10.**par_Ca[10])
        self.txtpCa2 = self.axProfCa1.text(-200*ff, 0.2, rf'$T$={temp:.1f}, $\xi$={xi:.1f}' ,fontsize=11  ) 

        self.legHa = self.axProfHa.legend()
        self.legCa = self.axProfCa.legend()
        self.figProf.tight_layout(pad=1)
        
        
#      Interactive Raster
        if self.Hapar and self.Capar:
            nd = 7   
            self.figIRaster, self.axIRaster = plt.subplots(2, nd, num='Parameter Maps', 
                                                sharex=True, sharey=True,    figsize=[10, 6],   clear=True)
            
            self.axIRaster = self.axIRaster.flatten()
                      
            
            for line in ['Ha', 'Ca']:
                if line == 'Ha':
                    pars = self.parHa
                    kind = 0
                else:
                    pars = self.parCa
                    kind = 1
                good = (pars[14] > 0.)* (pars[14]<0.015) *(pars[4] > (-0.15))     
                par0, psig = ThreeLayerScale(line)  
                for i in range(nd):               
                    j = ([4,  5, 12, 13,   9, 11, 14])[i]
                    name = ([rf'log $S_p$', rf'log $S_2$',  rf'log $S_1$', rf'log $S_0$', rf'$v_1$', 
                             rf'log $w_1$',  rf'$\epsilon$'])[i]
                    data = pars[j] 
 #                   if j == 14: data = np.log10(Geq(data, 0.03))
                    if j == 6:
                          data = np.log10(0.5*(10**pars[12]+10**pars[13]))
                    name1 = line+ ' '+name
                    if line == 'Ha': cmap=self.fissHa.cmap
                    if line == 'Ca': cmap=self.fissCa.cmap
                    if j == 8 or j == 9 : cmap='seismic'
                    tmp= self.axIRaster[kind*nd+i].imshow(data, cmap=cmap,
                                     origin='lower',  extent=extent,  interpolation=self.imInterp)
                    self.axIRaster[kind*nd+i].set_title(name1, fontsize=9)
                   
                    if j < 14:
                        m=data[good].mean()
                        sg=data[good].std()
                        tmp.set_clim(m-2.5*sg, m+2.5*sg)
                    else:
                        
                        tmp.set_clim(0, 0.015)
                    if j == 6:
                       
                        m = data[good].mean()
                        sg = data[good].std()
                        tmp.set_clim(m-3*sg, m+3*sg) 
 
            self.Iposition = []
            for ax in self.axIRaster:
                self.Iposition += [ax.scatter(self.x, self.y, 50, marker='+',
                                         color='c', linewidth=1)]
            self.figIRaster.tight_layout(pad=0.5)

    def _chWvHa(self):
        #Spectorgram
        self.vlineSpecHa.set_xdata(self.wvHa)
        self.axProfHa.set_title(r'$H\alpha$ %.2f $\AA$'%self.wvHa)
        #IRaster
        rasterHa = self.fissHa.getRaster(self.wvHa)
        whHa = rasterHa > 5
        if self.scale == 'log':
            rasterHa = np.log10(rasterHa)
            
    def _chWvCa(self):
        #Spectrogram
        self.vlineSpecCa.set_xdata(self.wvCa)
        #Profile
#        self.vlineProfCa.set_xdata(self.wvCa)
        self.axProfCa.set_title(r'Ca II %.2f $\AA$'%self.wvCa)
        #IRaster
        rasterCa = self.fissCa.getRaster(self.wvCa)
        whCa = rasterCa > 5
        if self.scale == 'log':
            rasterCa = np.log10(rasterCa)
        
    def _chX(self):
        # Spectrogram
        specHa = self.fissHa.data[:, self.xpix]
        specCa = self.fissCa.data[:, self.xpix, ::-1]
        if self.scale == 'log':
            specHa = np.log10(specHa)
            specCa = np.log10(specCa)
        self.imSpecHa.set_data(specHa)
        self.imSpecCa.set_data(specCa)
        self.axSpecHa.set_title(rf"H$\alpha$ X={self.x:.2f}'', Y={self.y:.2f}''"
                                r" (X$_{pix}$="f"{self.xpix}, "
                                r"Y$_{pix}$="f"{self.ypix})")
        
        # Raster
        for i in range(len(self.position)):
#            self.vlineRaster[i].set_xdata(self.x)
            self.position[i].set_offsets([self.x, self.y])
            
        # IRaster
        if self.Hapar: 
          for i in range(len(self.Iposition)):
#            self.vlineRaster[i].set_xdata(self.x)
            self.Iposition[i].set_offsets([self.x, self.y])

        
    def _chY(self):
        #Spectrogram
        self.hlineSpecHa.set_ydata(self.y)
        self.hlineSpecCa.set_ydata(self.y)
        self.axSpecHa.set_title(rf"H$\alpha$ X={self.x:.2f}'', Y={self.y:.2f}''"
                                r" (X$_{pix}$="f"{self.xpix}, "
                                r"Y$_{pix}$="f"{self.ypix})")
        #Raster
        for i in range(len(self.position)):
            self.position[i].set_offsets([self.x, self.y])
            
        # IRaster
        if self.Hapar: 
          for i in range(len(self.Iposition)):
            self.Iposition[i].set_offsets([self.x, self.y])

    def _chProfile(self):
        wvA1=self.fissHa.wave
        wvA = wvA1 + self.wvHaline
        wvB1=self.fissCa.wave
        wvB= wvB1+ self.wvCaline

        
        profA0=self.fissHa.data[self.ypix, self.xpix]
        profB0=self.fissCa.data[self.ypix, self.xpix]
        
        pureA=fpure(wvA, line='Ha')
 #       if self.Hapar: print( 'Ha  epsilon=', self.parHa[13, self.ypix, self.xpix])        
        profA = FissStray(pureA, wvA1, profA0, self.fissHa.refProfile)  
        parA,  modelA, modelpHa, fiterrorA, modelcHa \
            =  ThreeLayerModel(wvA, profA,  self.wvHaline, pureA, line='Ha')
#        tt2=time()    
        pureB=fpure(wvB, line='Ca')
#        if self.Capar: par_Ca=self.parCa[0:13, self.ypix, self.xpix]
        profB = FissStray(pureB, wvB1, profB0, self.fissCa.refProfile)  
        par_Ca,  modelB, modelpCa, fiterrorB, modelcCa \
            =  ThreeLayerModel(wvB, profB,  self.wvCaline, pureB,  line='Ca')

        self.pProfHa.set_ydata(profA)
        self.pProfHaa.set_ydata(modelpHa)
        self.pProfHab.set_ydata(modelcHa)
        self.pProfHac.set_ydata(modelA)        
        self.pProfCa.set_ydata(profB)
        self.pProfCaa.set_ydata(modelpCa)
        self.pProfCab.set_ydata(modelcCa)
#        self.pProfCac.set_xdata(wvB)
        self.pProfCac.set_ydata(modelB) 
#        self.pProfCad.set_ydata(fmodelB)        

        self.pProfHa1a.set_ydata(profA/modelcHa-1.)
        self.pProfCa1a.set_ydata(profB/modelcCa-1.)
        self.pProfCa1b.set_ydata(modelB/modelcCa-1.)
#        r1, r2 = fabsorp(wvB1, par_Ca, self.wvCaline)
#        self.pProfCa1c.set_ydata(0.1*r1)
#        self.pProfCa1d.set_ydata(0.1*r2)
        self.pProfHa1b.set_ydata(modelA/modelcHa-1.)
#        r1, r2 = fabsorp(wvA1, parA, self.wvHaline)
#        self.pProfHa1c.set_ydata(0.1*r1)
#        self.pProfHa1d.set_ydata(0.1*r2)


        self.pProfHa.set_label(f"data")
        self.pProfCa.set_label(f"data")
        self.legHa.remove()
        self.legHa = self.axProfHa.legend()
        self.legCa.remove()
        self.legCa = self.axProfCa.legend()

        self.txtpHa.set_text(  
                 rf'$x$={self.x:.1f}, $y$={self.y:.1f} Mm;  $v_p$={parA[0]:.2f} km/s;'+
                 rf'$S_p$={10**parA[4]:.2f}, $S_2$={10**(parA[5]):.2f}')
        self.txtpHaa.set_text( rf'log $\eta$={parA[1]:.2f}, log $w_p$={parA[2]:.2f}; log $a$={parA[3]:.2f}')                  

        self.txtpHa1.set_text(
           rf'$v_2$={parA[8]:.2f}, $v_1$={parA[9]:.2f} km/s; $w_2$={100*10**(parA[10]):.0f}, '+
           rf'$w_1$={100*10.**parA[11]:.0f} pm')  
        self.txtpHa1a.set_text(
           rf'$\tau_2$={10**parA[6]:.1f}, $\tau_1$={10**parA[7]:.1f}, $S_1$={10**parA[12]:.2f}, $S_0$={10**parA[13]:.2f}; $\epsilon$={fiterrorA:.3f}')
        self.txtpCa.set_text(
           rf'$x$={self.x:.1f}, $y$={self.y:.1f} Mm;  $v_p$={par_Ca[0]:.2f} km/s'+
           rf'; $S_p$={10**par_Ca[4]:.2f}, $S_2$={10**(par_Ca[5]):.2f} ' )
        self.txtpCaa.set_text(    
            rf'log $\eta$={par_Ca[1]:.2f}, log $w_p$={par_Ca[2]:.2f}; log $a$={par_Ca[3]:.2f}')
                  
        self.txtpCa1.set_text(          
            rf'$v_2$={par_Ca[8]:.2f}, $v_1$={par_Ca[9]:.2f} km/s; $w_2$={100*10.**(par_Ca[10]):.0f}, '+
            rf'$w_1$={1000*0.1*10.**par_Ca[11]:.0f} pm')
        self.txtpCa1a.set_text(  
            rf'$\tau_2$={10**par_Ca[6]:.1f}, $\tau_1$={10**par_Ca[7]:.1f}, $S_1$={10**par_Ca[12]:.2f}, $S_0$={10**par_Ca[13]:.2f};  $\epsilon$={fiterrorB:.3f}')
 
        temp, xi=ftandxi(10.**parA[11], 10.**par_Ca[11])
        self.txtpCa2.set_text(rf'$T$={temp:.1f}, $\xi$={xi:.1f}')

       
    def _onKey(self, event):
        
        if self.Hapar: 
            option = (event.inaxes in self.axIRaster)
        else:
            option = False
        if event.key == 'ctrl+right':
            if self.xpix < self.nx:
                self.xpix += 1
            else:
                self.xpix = 0
            self.x = (self.xpix+0.5-self.nx/2.)*self.xDelt*0.725
            self._chX()
            self._chProfile()
            self.xpixb = self.xpix0
            self.xb = self.x0
            self.xpix0 = self.xpix
            self.x0 = self.x
            
        elif event.key == 'ctrl+left':
            if self.xpix > 0:
                self.xpix -= 1
            else:
                self.xpix = self.nx
            self.x = (self.xpix+0.5-self.nx/2.)*self.xDelt*0.725
            self._chX()
            self._chProfile()
            self.xpixb = self.xpix0
            self.xb = self.x0
            self.xpix0 = self.xpix
            self.x0 = self.x
        elif event.key == 'ctrl+up':
            if self.ypix < self.ny:
                self.ypix += 1
            else:
                self.ypix = 0
            self.y = (self.ypix+0.5-self.ny/2.)*self.yDelt*0.725
            self._chY()
            self._chProfile()
            self.ypixb = self.ypix0
            self.yb = self.y0
            self.ypix0 = self.ypix
            self.y0 = self.y
        elif event.key == 'ctrl+down':
            if self.ypix > 0:
                self.ypix -= 1
            else:
                self.ypix = self.ny
            self.y = (self.ypix+0.5-self.ny/2.)*self.yDelt*0.725
            self._chY()
            self._chProfile()
            self.ypixb = self.ypix0
            self.yb = self.y0
            self.ypix0 = self.ypix
            self.y0 = self.y
        elif event.key == 'left':
            if self.wvHa > self.fissHa.wave.min():
                self.wvHa -= abs(self.fissHa.wvDelt)
            else:
                self.wvHa = self.fissHa.wave.max()
            self.wvHab = self.wvHa0
            self._chWvHa()
            self.wvHa0 = self.wvHa
        elif event.key == 'right':
            if self.wvHa < self.fissHa.wave.max():
                self.wvHa += abs(self.fissHa.wvDelt)
            else:
                self.wvHa = self.fissHa.wave.min()
            self.wvHab = self.wvHa0
            self._chWvHa()
            self.wvHa0 = self.wvHa
        elif event.key == 'down':
            if self.wvCa > self.fissCa.wave.min():
                self.wvCa -= abs(self.fissCa.wvDelt)
            else:
                self.wvCa = self.fissCa.wave.max()
            self.wvCab = self.wvCa0
            self._chWvCa()
            self.wvCa0 = self.wvCa
        elif event.key == 'up':
            if self.wvCa < self.fissCa.wave.max():
                self.wvCa += abs(self.fissCa.wvDelt)
            else:
                self.wvCa = self.fissCa.wave.min()
            self.wvCab = self.wvCa0
            self._chWvCa()
            self.wvCa0 = self.wvCa
        elif event.key == ' ' and \
             ((event.inaxes in self.axRaster) or option):

            x = event.xdata
            y = event.ydata
            self.xpix = int(round(x/(self.xDelt*.725)+self.nx/2.-0.5))
            self.x = (self.xpix+0.5-self.nx/2.)*self.xDelt*0.725
            self.ypix=  int(round(y/(self.yDelt*0.725)+self.ny/2-0.5))
            self.y = (self.ypix+0.5-self.ny/2.)*self.yDelt*0.725
#            self.x = self.xpix*self.xDelt+self.xDelt/2
#            self.y = self.ypix*self.yDelt+self.yDelt/2
            self._chX()
            self._chY()
            self._chProfile()
            self.xpixb = self.xpix0
            self.xb = self.x0
            self.ypixb = self.ypix0
            self.yb = self.y0
            self.xpix0 = self.xpix
            self.x0 = self.x
            self.ypix0 = self.ypix
            self.y0 = self.y
        elif event.key == ' ' and event.inaxes == self.axProfHa:
            self.wvHa = event.xdata
            self.wvHab = self.wvHa0
            self._chWvHa()
            self.wvHa0 = self.wvHa
        elif event.key == ' ' and event.inaxes == self.axProfCa:
            self.wvCa = event.xdata
            self.wvCab = self.wvCa0
            self._chWvCa()
            self.wvCa0 = self.wvCa
        elif event.key == ' ' and event.inaxes == self.axSpecHa:
            self.wvHa = event.xdata
            y = event.ydata
#            self.ypix = int(round((y-self.yDelt/2)/self.yDelt))
#            self.y = self.ypix*self.yDelt+self.yDelt/2
            self.ypix=  int(round(y/(self.yDelt*0.725)+self.ny/2-0.5))
            self.y = (self.ypix+0.5-self.ny/2.)*self.yDelt*0.725
            self._chWvHa()
            self._chY()
            self._chProfile()
            self.wvHab = self.wvHa0
            self.wvHa0 = self.wvHa
            self.ypixb = self.ypix0
            self.yb = self.y0
            self.ypix0 = self.ypix
            self.y0 = self.y
        elif event.key == ' ' and event.inaxes == self.axSpecCa:
            self.wvCa = event.xdata
            y = event.ydata
#            self.ypix = int(round((y-self.yDelt/2)/self.yDelt))
#           self.y = self.ypix*self.yDelt+self.yDelt/2
            self.ypix=  int(round(y/(self.yDelt*0.725)+self.ny/2-0.5))
            self.y = (self.ypix+0.5-self.ny/2.)*self.yDelt*0.725

            self._chWvCa()
            self._chY()
            self._chProfile()
            self.wvCab = self.wvCa0
            self.wvCa0 = self.wvCa
            self.ypixb = self.ypix0
            self.yb = self.y0
            self.ypix0 = self.ypix
            self.y0 = self.y
        elif event.key == 'ctrl+b':
            wvHa = self.wvHab
            self.wvHab = self.wvHa
            self.wvHa = self.wvHa0 = wvHa
            wvCa = self.wvCab
            self.wvCab = self.wvCa
            self.wvCa = self.wvCa0 = wvCa
            x = self.xb
            xpix = self.xpixb
            self.xb = self.x
            self.xpixb = self.xpix
            self.x = self.x0 = x
            self.xpix = self.xpix0 = xpix
            y = self.yb
            ypix = self.ypixb
            self.yb = self.y
            self.ypixb = self.ypix
            self.y = self.y0 = y
            self.ypix = self.ypix0 = ypix
            self._chX()
            self._chY()
            self._chProfile()
            self._chWvHa()
            self._chWvCa()
        
        if self.Hapar: self.figIRaster.canvas.draw_idle()
        self.figProf.canvas.draw_idle()
        self.figRaster.canvas.draw_idle()
        self.figSpec.canvas.draw_idle()
        
    def _helpShow(self):
        helpBox = plt.figure('Interactive Key', figsize=[3.5, 3])
        ax = helpBox.add_subplot(111)
        ax.set_position([0,0,1,1])
        ax.text(0.05, 0.92, 'ctrl+b: Back to Previous Position')
        ax.text(0.05,0.82,'ctrl+right: Move to right')
        ax.text(0.05,0.72,'ctrl+left: Move to left')
        ax.text(0.05,0.62,'ctrl+up: Move to up')
        ax.text(0.05,0.52,'ctrl+down: Move to down')
        ax.text(0.05,0.42,'right: Increase the wavelength of the fissHa')
        ax.text(0.05,0.32,'left: Decrease the wavelength of the fissHa')
        ax.text(0.05,0.22,'up: Increase the wavelength of the fissCa')
        ax.text(0.05,0.12,'down: Decrease the wavelength of the fissCa')
        ax.text(0.05,0.02,'spacebar: CHange to current mouse point')
        
    def get_profileHa(self, xpix=None, ypix=None):
        """
        Get line profile of the Ha band.
        
        Parameters
        ----------
        xpix : int
            x pixel position.
            Default is `~lab2.xpix`
        ypix : int
            y pixel position.
            Default is `~lab2.ypix`
        """
        if not xpix:
            xpix = self.xpix
        if not ypix:
            ypix = self.ypix
        return self.fissHa.data[ypix, xpix]
        
    def get_profileCa(self, xpix=None, ypix=None):
        """
        Get line profile of the Ca II band.
        
        Parameters
        ----------
        xpix : int
            x pixel position.
            Default is `~lab2.xpix`
        ypix : int
            y pixel position.
            Default is `~lab2.ypix`
        """
        if not xpix:
            xpix = self.xpix
        if not ypix:
            ypix = self.ypix
        return self.fissCa.data[ypix, xpix]
    
    
    def do_fits(self, x, y, line='Ha', logtau0=1.0):
       
        n=x.size
        wvpHa = 6559.580
        wvcHa = 6562.817
        wvpCa = 8536.165
        wvcCa = 8542.091
 
        wvA=self.fissHa.wave+wvcHa
        wvB=self.fissCa.wave+wvcCa

        epsilon=0.027
        
        for i in range(n):
            xpix=x[i]
            ypix=y[i]
#            print('xpix=', xpix, ', ypix=', ypix)
            if line == 'Ha':
                profA0=(self.fissHa.data[ypix,xpix]-epsilon*self.fissHa.refProfile)  \
                  /((1-epsilon)*self.fissHa.refProfile.max())        
                profA,intcA, pHa, pA, modelA, cHa, cA, cmodelA, fmodelA, fiterror \
                                =  DecompTwo(wvA, profA0, self.wvHaline, logtau0=logtau0)
                self.parHa[:,ypix,xpix]=np.append(np.append(pA, cA[0:7]), fiterror)
            elif line =='Ca':    
                profB0=(self.fissCa.data[ypix,xpix]-epsilon*self.fissCa.refProfile) \
                    /((1-epsilon)*self.fissCa.refProfile.max())

  #              profB0=self.fissCa.data[ypix, xpix]/self.fissCa.refProfile.max()
                profB,intcB,  pCa, pB, modelB, cCa, cB, cmodelB, fmodelB, fiterror \
                                =  DecompTwo(wvB, profB0, self.wvCaline, line='Ca',  logtau0=logtau0)
                self.parCa[:,ypix,xpix]=np.append(np.append(pB, cB[0:7]), fiterror)
    
            
    def do_fitR(self, x1,x2, y1,y2, line='Ha', logtau0=1.0):
        
#        print('logtau0=', logtau0)
        npoint=(x2-x1)*(y2-y1)
        xindex=(np.ones(y2-y1, int))[:,None]*np.arange(x1,x2)
        yindex=(np.arange(y1,y2))[:,None]*np.ones(x2-x1,int)
        xindex=xindex.resHape(npoint)
        yindex=yindex.resHape(npoint)
        self.do_fits(xindex, yindex, line=line, logtau0=logtau0)